##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# REINFORCE 에이전트

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 소개

이 예제는 [DQN 튜토리얼](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)과 마찬가지로, TF-Agents 라이브러리를 사용하여 Cartpole 환경에서 [REINFORCE](1_dqn_tutorial.ipynb) 에이전트를 훈련하는 방법을 보여줍니다.

![카트 폴 환경](images/cartpole.png)

훈련, 평가 및 데이터 수집을 위한 RL(Reinforcement Learning) 파이프라인의 모든 구성 요소를 설명합니다.


## 설정

다음과 같은 종속성을 설치하지 않은 경우, 다음을 실행합니다.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet xvfbwrapper


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## 하이퍼 매개변수

In [ ]:
env_name = "CartPole-v0" # @param {type:"string"}
num_iterations = 250 # @param {type:"integer"}
collect_episodes_per_iteration = 2 # @param {type:"integer"}
replay_buffer_capacity = 2000 # @param {type:"integer"}

fc_layer_params = (100,)

learning_rate = 1e-3 # @param {type:"number"}
log_interval = 25 # @param {type:"integer"}
num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

## 환경

RL의 환경은 해결하려는 작업 또는 문제를 나타냅니다. `suites`를 사용하여 TF-Agents에서 표준 환경을 쉽게 작성할 수 있습니다. 문자열 환경 이름을 고려하여 OpenAI Gym, Atari, DM Control 등과 같은 소스에서 환경을 로드하기 위한 서로 다른 `suites`가 있습니다.

이제 OpenAI Gym 도구 모음에서 CartPole 환경을 로드하겠습니다.

In [ ]:
env = suite_gym.load(env_name)

이 환경을 렌더링하여 형태를 볼 수 있습니다. 자유로 흔들리는 막대가 카트에 부착되어 있습니다. 목표는 막대가 위를 향하도록 카트를 오른쪽이나 왼쪽으로 이동하는 것입니다.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

`time_step = environment.step(action)` 문은 환경에서 `action`를 취합니다. 반환된 `TimeStep` 튜플에는 환경의 다음 관찰 값 및 해당 행동에 대한 보상이 포함됩니다. 환경에서 `time_step_spec()` 및 `action_spec()` 메서드는 `time_step` 및 `action` 각각의 사양(types, shapes, bounds)을 반환합니다.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

따라서 관찰 값은 4개의 부동 소수점, 카트의 위치와 속도, 막대의 각 위치와 속도의 배열입니다. 두 가지 행동(왼쪽으로 이동 또는 오른쪽으로 이동)만 가능하므로 `action_spec`은 스칼라이며, 0은 "왼쪽으로 이동"을 의미하고 1은 "오른쪽으로 이동"을 의미합니다.

In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

일반적으로, 훈련용 환경과 평가용 환경의 두 가지 환경을 만듭니다. 대부분의 환경은 순수한 Python으로 작성되지만, `TFPyEnvironment` 래퍼를 사용하여 TensorFlow로 쉽게 변환할 수 있습니다. 원래 환경의 API는 numpy 배열을 사용하고, `TFPyEnvironment`는 TensorFlow 정책 및 에이전트와 보다 쉽게 ​​상호 작용할 수 있도록 numpy 배열을 `Tensors`로/에서 변환합니다.


In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## 에이전트

RL 문제를 해결하기 위해 사용하는 알고리즘은 `Agent`로 표시됩니다. REINFORCE 에이전트 외에도, TF-Agents는 [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf), [DDPG](https://arxiv.org/pdf/1509.02971.pdf), [TD3](https://arxiv.org/pdf/1802.09477.pdf), [PPO](https://arxiv.org/abs/1707.06347) 및 [SAC](https://arxiv.org/abs/1801.01290)와 같은 다양한 `Agents`의 표준 구현을 제공합니다.

REINFORCE 에이전트를 만들려면 먼저 환경의 관찰 값을 고려하여 학습을 통해 행동을 예측할 수 있는 `Actor Network`가 필요합니다.

관찰 값 및 행동의 사양을 사용하여 `Actor Network`를 쉽게 작성할 수 있습니다. 네트워크에서 레이어를 지정할 수 있습니다. 이 예제에서 레이어는 숨겨진 각 레이어의 크기를 나타내는 `ints`의 튜플로 설정된 `fc_layer_params` 인수입니다(위의 하이퍼 매개변수 섹션 참조).


In [ ]:
actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

또한, 방금 생성한 네트워크를 훈련하기 위해 `optimizer`가 필요하고, 네트워크가 몇 번 업데이트되었는지 추적하기 위해`train_step_counter` 변수가 필요합니다.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = reinforce_agent.ReinforceAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
tf_agent.initialize()

## 정책

TF-Agents에서 정책은 RL의 표준 정책 개념을 나타냅니다. 즉, `time_step`이 주어지면 행동 또는 행동에 대한 분포를 생성합니다. 기본 메서드는 `policy_step = policy.action(time_step)`이고, 여기서 `policy_step`은 명명된 튜플 `PolicyStep(action, state, info)`입니다. `policy_step.action`은 환경에 적용할 `action`이고 `state`는 상태 저장(RNN) 정책에 대한 상태를 나타내며 `info`에는 동작의 로그 확률 등의 보조 정보가 포함될 수 있습니다.

에이전트에는 평가/배포에 사용되는 기본 정책(agent.policy)과 데이터 수집에 사용되는 또 다른 정책(agent.collect_policy)의 두 가지 정책이 있습니다.

In [ ]:
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

## 메트릭 및 평가

정책을 평가하는 데 사용되는 가장 일반적인 메트릭은 평균 이익입니다. 이익은 에피소드의 환경에서 정책을 실행하는 동안 얻은 보상의 합계이며 일반적으로 몇 개 에피소드에서 평균으로 계산합니다. 다음과 같이 평균 이익 메트릭을 계산할 수 있습니다.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# Please also see the metrics module for standard implementations of different
# metrics.

## 재현 버퍼

환경에서 수집된 데이터를 추적하기 위해 Deepmind의 효율적이고 확장 가능하며 사용하기 쉬운 재생 시스템인 [Reverb](https://deepmind.com/research/open-source/Reverb)를 사용합니다. Reverb는 Trajectory를 수집할 때 경험 데이터를 저장하기도 하지만 훈련 중에는 소비되기도 합니다.

이 재현 버퍼는 저장될 텐서를 설명하는 사양을 사용하여 구성되며 `tf_agent.collect_data_spec`을 사용하여 에이전트에서 가져올 수 있습니다.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      tf_agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    table_name=table_name,
    sequence_length=None,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddEpisodeObserver(
    replay_buffer.py_client,
    table_name,
    replay_buffer_capacity
)

에이전트 대부분의 경우, `collect_data_spec`은 관찰 값, 행동, 보상 등을 포함하는 `Trajectory` 명명된 튜플입니다.

## 데이터 수집

As REINFORCE learns from whole episodes, we define a function to collect an episode using the given data collection policy and save the data (observations, actions, rewards etc.) as trajectories in the replay buffer. Here we are using 'PyDriver' to run the experience collecting loop. You can learn more about TF Agents driver in our [drivers tutorial](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}

def collect_episode(environment, policy, num_episodes):

  driver = py_driver.PyDriver(
    environment,
    py_tf_eager_policy.PyTFEagerPolicy(
      policy, use_tf_function=True),
    [rb_observer],
    max_episodes=num_episodes)
  initial_time_step = environment.reset()
  driver.run(initial_time_step)

## 에이전트 훈련하기

훈련 루프에는 환경에서 데이터를 수집하고 에이전트의 네트워크를 최적화하는 것이 포함됩니다. 그 과정에서 에이전트의 정책을 평가하여 진행 상황을 파악할 수 있습니다.

다음은 실행하는 데 약 3분이 걸립니다.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(
      train_py_env, tf_agent.collect_policy, collect_episodes_per_iteration)

  # Use data from the buffer and update the agent's network.
  iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  trajectories, _ = next(iterator)
  train_loss = tf_agent.train(experience=trajectories)  

  replay_buffer.clear()

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## 시각화


### 플롯

에이전트의 성능을 확인하기 위해 이익 vs 글로벌 스텝을 플롯할 수 있습니다. `Cartpole-v0`에서 환경은 막대가 위로 유지되는 타임스템마다 +1의 보상을 제공하며, 최대 스텝 수는 200개이므로 가능한 최대 이익도 200입니다.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=250)

### 비디오

각 단계에서 환경을 렌더링하여 에이전트의 성능을 시각화하는 것이 도움이 됩니다. 이를 수행하기 전에, 먼저 이 Colab에 비디오를 포함하는 함수를 작성합니다.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

다음 코드는 몇 가지 에피소드에 대한 에이전트 정책을 시각화합니다.

In [ ]:
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = tf_agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)